### An Improved Coding Assistant with LangChain, Ollama and Function Calling


In [ ]:
# Install libraries
%pip install -qU langchain
%pip install -qU langchain_ollama
%pip install -qU langchain_core
%pip install -qU html2text

In [36]:
# Import necessary libraries
import requests
import html2text

from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

from langchain.globals import set_verbose, set_debug
set_verbose(True)
set_debug(True)

### Tool definitions


In [37]:
@tool
def get_architectural_guidelines(componentName: str, location: str) -> str:
    """Returns architectural guidelines for a React project.

    Args:
        componentName: component name
        location: URL of the architectural guidelines
    """
    
    print(f"Tool call: Getting architectural guidelines for {componentName} from {location}")
    response = requests.get(location)
    html_content = response.text
    return html2text.html2text(html_content)

### Model definition


In [38]:
# Initialize the local model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

### Bind tools to model


In [39]:
tools = [get_architectural_guidelines]
llm_with_tools = llm.bind_tools(tools)

### Build messages


In [40]:
# System message
systemMessage = "You are a software developer specialized in React."
messages = [SystemMessage(systemMessage)]

# Human message
humanMessage = "Create a react component Sign Up Form for an online banking app. Use best practices and the architectural guidelines from http://localhost:8080/data/archguide.html"
messages.append(HumanMessage(humanMessage))

### Call the model with System + Human messages


In [ ]:
ai_msg = llm_with_tools.invoke(messages)

### Append tool response to messages


In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_architectural_guidelines": get_architectural_guidelines}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

### Call the model with System + Human + Tool messages


In [ ]:
ai_msg = llm_with_tools.invoke(messages)

### Print the final results


In [ ]:
print(ai_msg.content)